In [ ]:
import json
import glob
import os

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, select, func, and_, insert, delete, update, or_

from tqdm import tqdm

In [ ]:
annotation_paths = glob.glob("/root/data/gtsf_phase_I/*/*.json")
annotation_paths

In [ ]:
# annotations = []
# for jpath in annotation_paths:
#     annotations += json.load(open(jpath))
# print("Number of validation data: {}".format(len(annotations)))

In [ ]:
for jpath in annotation_paths:
    annotations = json.load(open(jpath))
    print(jpath)
    print("Amount of data: {}".format(len(annotations)))
    
    sql_credentials = json.load(open("/root/thomas/sql_research_credentials.json"))

    sql_engine = create_engine(
        "postgresql://{}:{}@{}:{}/{}".format(sql_credentials["user"], sql_credentials["password"],
                                             sql_credentials["host"], sql_credentials["port"],
                                             sql_credentials["database"]))

    metadata = MetaData()
    gtsf = Table('gtsf_data_collections', metadata, autoload=True, autoload_with=sql_engine)

    for ann in annotations:
        local_path = os.path.join("/root/data/gtsf_phase_I/", 
                      "/".join(ann["Labeled Data"].split("/")[7:]))
        ann["local_path"] = local_path
        if not os.path.isfile(local_path):
            print("missing image!!")
        for body_part in ann["Label"].keys():
            new_body_part = "_".join(body_part.replace(":", "").split()).upper()
            ann["Label"][new_body_part] = ann["Label"].pop(body_part)

    timestamps = []
    for ann in annotations:
        timestamp = ann["local_path"].split("/")[-3]
        ann["timestamp"] = timestamp
        timestamps.append(ann["timestamp"])

    query = select([gtsf.c.ground_truth_metadata,
                    gtsf.c.gtsf_fish_identifier]).select_from(gtsf).where(gtsf.c.gtsf_fish_identifier.in_(timestamps))
    connection = sql_engine.connect()
    q = connection.execute(query)
    results = [(eval(r[0]), r[1]) for r in q]

    for ann in annotations:
        for r in results:
            if r[1] == ann["timestamp"]:
                ann["weight"] = r[0]["data"]["weight"]
                ann["breath"] = r[0]["data"]["breath"]
                ann["length"] = r[0]["data"]["length"]
                ann["width"] = r[0]["data"]["width"]
                ann["kfactor"] = 10**5*ann["weight"] / ann["length"]**3
                ann["species"] = r[0]["data"].get("species", "salmon")
                break
    
    with open(jpath, "w") as f:
        json.dump(annotations, f)